In [20]:
import logging
import sys
from normstats import *
np.set_printoptions(suppress=True)

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(funcName)s - %(levelname)s - %(message)s',
    stream=sys.stdout,
    force=True
)
logger = logging.getLogger(__name__)

# For reproducibility
np.random.seed(abs(hash("giraffes are cool"))%(2**32 - 1))

In [53]:
# Parameters for normative synthetic data generation

# number of samples
n = 50
# number of ind vars
k = 1
# number of dep vars
m = 1
# std_dev of noise
S_YdotX_actual = np.random.uniform(0.01, 0.02, size=(m))

# B (model params) will be chosen randomly from this range
B_range = (1000, 1005)
#B_actual = np.random.randint(B_range[0], B_range[1], size=(m, k+1))
B_actual = np.random.uniform(B_range[0], B_range[1], size=(m, k+1))
        
# X (ind vars) will be chosen randomly from this range
X_range = (1,20)
X_are_ints = True

In [54]:
# Generate synthetic normative dataset
X, Y, epsilon_actual = gen_synth_norm_data(B_actual, n, S_YdotX_actual, X_range, X_are_ints=X_are_ints)

# estimate model params from synthetic dataset
B_estimate, S_YdotX_estimate, R, X_mean, X_std = estimate_model_params(X,Y)

2025-05-28 15:17:22,592 - normstats - gen_synth_norm_data - DEBUG - n: 50
2025-05-28 15:17:22,592 - normstats - gen_synth_norm_data - DEBUG - X_are_ints: True
2025-05-28 15:17:22,593 - normstats - gen_synth_norm_data - DEBUG - m: 1
2025-05-28 15:17:22,593 - normstats - gen_synth_norm_data - DEBUG - k: 1
2025-05-28 15:17:22,593 - normstats - gen_synth_norm_data - DEBUG - X.shape: (1, 50)
2025-05-28 15:17:22,594 - normstats - gen_synth_norm_data - DEBUG - S_YdotX.shape: (1,)
2025-05-28 15:17:22,594 - normstats - z_normalize - DEBUG - X.shape: (1, 50)
2025-05-28 15:17:22,594 - normstats - z_normalize - DEBUG - n: 50
2025-05-28 15:17:22,595 - normstats - z_normalize - DEBUG - X_mean.shape: (1, 1)
2025-05-28 15:17:22,595 - normstats - z_normalize - DEBUG - X_std.shape: (1, 1)
2025-05-28 15:17:22,596 - normstats - z_normalize - DEBUG - Z.shape: (1, 50)
2025-05-28 15:17:22,596 - normstats - z_normalize - DEBUG - Z_1.shape: (2, 50)
2025-05-28 15:17:22,597 - normstats - gen_synth_norm_data - DE

In [55]:
print('B_actual:')
print(B_actual)
print()
print('B_estimate')
print(B_estimate)
print()
print('S_YdotX_actual:')
print(S_YdotX_actual)
print()
print('S_YdotX_estimate:')
print(S_YdotX_estimate)

B_actual:
[[1004.21703588 1002.62968104]]

B_estimate
[[1004.21933473 1002.63074176]]

S_YdotX_actual:
[0.01723041]

S_YdotX_estimate:
[[0.01564461]]


In [56]:
S_YdotX_actual.shape

(1,)

In [57]:
# Generate a single perfectly median subject
x_obs, y_obs = gen_synth_sub_data(B_actual, 1, 0.0, X_range, X_mean, X_std, X_are_ints=X_are_ints)

2025-05-28 15:17:31,543 - normstats - gen_synth_sub_data - DEBUG - X_are_ints: True
2025-05-28 15:17:31,543 - normstats - gen_synth_sub_data - DEBUG - n: 1
2025-05-28 15:17:31,544 - normstats - gen_synth_sub_data - DEBUG - m: 1
2025-05-28 15:17:31,544 - normstats - gen_synth_sub_data - DEBUG - k: 1
2025-05-28 15:17:31,545 - normstats - z_normalize - DEBUG - X.shape: (1, 1)
2025-05-28 15:17:31,545 - normstats - z_normalize - DEBUG - n: 1
2025-05-28 15:17:31,545 - normstats - z_normalize - DEBUG - X_mean.shape: (1, 1)
2025-05-28 15:17:31,545 - normstats - z_normalize - DEBUG - X_std.shape: (1, 1)
2025-05-28 15:17:31,545 - normstats - z_normalize - DEBUG - Z.shape: (1, 1)
2025-05-28 15:17:31,546 - normstats - z_normalize - DEBUG - Z_1.shape: (2, 1)


In [58]:
p, p_ci = single_subject_eval(x_obs, y_obs, B_estimate, R, S_YdotX_estimate, n, X_mean, X_std)

2025-05-28 15:17:32,839 - normstats - single_subject_eval - DEBUG - m: 1
2025-05-28 15:17:32,840 - normstats - single_subject_eval - DEBUG - k: 1
2025-05-28 15:17:32,840 - normstats - z_normalize - DEBUG - X.shape: (1, 1)
2025-05-28 15:17:32,840 - normstats - z_normalize - DEBUG - n: 1
2025-05-28 15:17:32,841 - normstats - z_normalize - DEBUG - X_mean.shape: (1, 1)
2025-05-28 15:17:32,841 - normstats - z_normalize - DEBUG - X_std.shape: (1, 1)
2025-05-28 15:17:32,841 - normstats - z_normalize - DEBUG - Z.shape: (1, 1)
2025-05-28 15:17:32,841 - normstats - z_normalize - DEBUG - Z_1.shape: (2, 1)
2025-05-28 15:17:32,842 - normstats - single_subject_eval - DEBUG - r_A: 0.0007731751281916654
2025-05-28 15:17:32,842 - normstats - single_subject_eval - DEBUG - r_B: 0.0
2025-05-28 15:17:32,842 - normstats - single_subject_eval - DEBUG - S_Nplus1: [[0.0158004]]
2025-05-28 15:17:32,843 - normstats - single_subject_eval - DEBUG - t_diff: [[-0.09573926]]
2025-05-28 15:17:32,843 - normstats - sing

In [59]:
# Should be ~0.5 since it's a perfectly median subject
print('p')
print(p[:,0])
print('lower confidence interval of p')
print(p_ci[:,0])
print('upper confidence interval of p')
print(p_ci[:,1])

p
[0.46205907]
lower confidence interval of p
[0.35414651]
upper confidence interval of p
[0.57210814]
